In [ ]:
import joblib

from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
import optuna
study = optuna.study.load_study('no-name-2381af36-3492-486c-b2b5-e4ebe82eeb83')
# joblib.dump(study, "study_5_IMGS_AVG_OBJ.pkl")

import joblib
study = joblib.load("study_5_IMGS_AVG_OBJ.pkl")
print("Best trial until now:")
print(" Value: ", study.best_trial.value)
print(" Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")
    
plot_optimization_history(study, target=lambda t:np.mean(t.values[1]))

plot_contour(study)

plot_contour(study, target=lambda t:np.mean(t.user_attrs['r']), target_name='r')

plot_contour(study, target=lambda t:np.log(np.mean(t.user_attrs['num_nodes'])), target_name='log num_nodes')

plot_contour(study, target=lambda t:np.std(t.user_attrs['r']), target_name='std r')

plot_contour(study, target=lambda t:np.std(t.user_attrs['num_nodes']), target_name='std num_nodes')

plot_slice(study,target=lambda t:np.mean(t.user_attrs['r']), target_name='r')

plot_slice(study,target=lambda t:np.mean(t.user_attrs['num_nodes']), target_name='num_nodes')

optuna.visualization.plot_pareto_front(study, target_names=["r", "num_nodes"])

with open('10IMGS_optuna_multi_obj.txt', 'r') as f2:
    arr = [[float(x) for x in t.strip().split()] for t in f2.readlines()]
len(arr)


num_nodes_lst = []
r_lst = []
thresh_lst  = []
cnt_area_max_lst = []

for num_nodes, r, thresh, cnt_area_max in arr:
    num_nodes_lst.append(num_nodes)
    r_lst.append(r)
    thresh_lst.append(thresh)
    cnt_area_max_lst.append(cnt_area_max)
    
    
pareto_front = np.zeros(len(arr), dtype=bool)
par = []
for i in range(len(arr)):
    dom = False
    for j in range(len(arr)):
        if i != j and arr[j][0] < arr[i][0] and arr[j][1] > arr[i][1]: # pt. j dominates pt. i if num_nodes_j < num_nodes_i and r_j > r_i
            dom = True
    if not dom: #pareto front
        pareto_front[i] = True
        par.append((arr[i][1], arr[i][0]))

        
plt.figure(dpi=200)
plt.scatter(x=r_lst, y=num_nodes_lst, s=30, c='blue',  edgecolors= "black", linewidth=0.5)
# plt.title('Min r among random 10 images per hyperparams evaluation')
arr2 = np.array(sorted(par, key = lambda t: t[0]))
plt.plot(arr2[:, 0], arr2[:, 1], c='red', label='pareto front')
for i in range(arr2.shape[0]):
    plt.text(arr2[i][0]+0.001, arr2[i][1]-500, f'({np.around(arr2[i][0],3)}, {int(arr2[i][1])})', fontsize=4, color='red')
plt.title('Trade-offs for varying (thresh, cnt_area_max) in the BFS algorithm')
plt.xlabel('min r among 20 imgs.')
plt.ylabel('max num_nodes among 20 imgs.')
plt.legend()        

from matplotlib import colors
divnorm=colors.TwoSlopeNorm(vmin=0., vcenter=0.9, vmax=1)
plt.figure(dpi=200)
for i in range(len(arr)):
    plt.scatter(x=thresh_lst[i],y=cnt_area_max_lst[i] , c=r_lst[i], s=30, cmap='seismic', norm=divnorm, edgecolors= "black", linewidth=0.5)
plt.title('Min r among random 20 images per hyperparams evaluation')
plt.xlabel('log thresh')
plt.xscale('log')
plt.ylabel('cnt_area_max')
plt.ylim(0, 200)
plt.colorbar(label='r')

from matplotlib import colors
divnorm=colors.TwoSlopeNorm(vmin=0., vcenter=7000, vmax=20000)
plt.figure(dpi=200)
for i in range(len(arr)):
    plt.scatter(x=thresh_lst[i],y=cnt_area_max_lst[i], c=num_nodes_lst[i], s=30, cmap='PiYG', edgecolors= "black", linewidth=0.5, norm=divnorm)

plt.title('Max num_nodes among random 20 images per hyperparams evaluation')
plt.xlabel('log thresh')
plt.ylabel('cnt_area_max')
plt.ylim(0, 200)
plt.xscale('log')
plt.colorbar(label='num_nodes')

In [ ]:
import optuna
NUM_IMG = 5
R_WEIGHT = 100000
def objective(trial):

    # Invoke suggest methods of a Trial object to generate hyperparameters.
    thresh = trial.suggest_loguniform('thresh', 0.0001, 1)
    cnt_area_max = trial.suggest_uniform('cnt_area_max',1, 200)
    r_lst, num_nodes_lst, chosen_idx_lst = [], [], []
    for _ in range(NUM_IMG):
        chosen_idx = np.random.randint(len(train_data))
        chosen_idx_lst.append(chosen_idx)
        
        img, msk, fn = train_data[chosen_idx]
        
        img = np.moveaxis(img.numpy(), 0, -1)
        msk_discrete = np.argmax(msk.numpy(), axis=0)
        dst = cv2.Canny(np.uint8(img*255), 50, 200, None, 3)
        dst = dst>0
        partition_res, mean_c_res, res_lst, cnt_res_lst, num_nodes = partition_image(dst, img, thresh=thresh, cnt_area_max=cnt_area_max)

        r, _, _ = avg_acc(num_nodes, partition_res, res_lst, msk_discrete)
        r_lst.append(r)
        num_nodes_lst.append(num_nodes)
        
    trial.set_user_attr("r", r_lst)
    trial.set_user_attr("num_nodes", num_nodes_lst)
    trial.set_user_attr("chosen_idx", chosen_idx_lst)
    
    obj = (np.sum(num_nodes_lst) - np.sum(r_lst)*R_WEIGHT)/NUM_IMG
    
    return obj

def logging_callback(study, frozen_trial):
#     study.set_user_attr("previous_best_value", study.best_value)
    print(
        "Trial {} finished with value: {} and parameters: {} | r = {}, num_nodes = {}, chosen_idx = {} ".format(
        frozen_trial.number,
        frozen_trial.value,
        frozen_trial.params,
        frozen_trial.user_attrs['r'],
        frozen_trial.user_attrs['num_nodes'],
        frozen_trial.user_attrs['chosen_idx']
        )
    )

study = optuna.create_study(direction="minimize")  # Create a new study.
study.optimize(objective, n_trials=100, callbacks=[logging_callback])  # Invoke optimization of the objective function.

In [ ]:
import optuna
NUM_IMG = 20
R_WEIGHT = 100000
def objective(trial):

    # Invoke suggest methods of a Trial object to generate hyperparameters.
    thresh = trial.suggest_loguniform('thresh', 0.0001, 1)
    cnt_area_max = trial.suggest_uniform('cnt_area_max',1, 200)
    r_lst, num_nodes_lst, chosen_idx_lst = [], [], []
    for _ in range(NUM_IMG):
        chosen_idx = np.random.randint(len(train_data))
        chosen_idx_lst.append(chosen_idx)
        
        img, msk, fn = train_data[chosen_idx]
        
        img = np.moveaxis(img.numpy(), 0, -1)
        msk_discrete = np.argmax(msk.numpy(), axis=0)
        dst = cv2.Canny(np.uint8(img*255), 50, 200, None, 3)
        dst = dst>0
        partition_res, mean_c_res, res_lst, cnt_res_lst, num_nodes = partition_image(dst, img, thresh=thresh, cnt_area_max=cnt_area_max)

        r, _, _ = avg_acc(num_nodes, partition_res, res_lst, msk_discrete)
        r_lst.append(r)
        num_nodes_lst.append(num_nodes)
        
    trial.set_user_attr("r", r_lst)
    trial.set_user_attr("num_nodes", num_nodes_lst)
    trial.set_user_attr("chosen_idx", chosen_idx_lst)
    
    return np.max(num_nodes_lst), np.min(r_lst)

def logging_callback(study, frozen_trial):
#     study.set_user_attr("previous_best_value", study.best_value)
    print(
        "Trial {} finished and parameters: {} | r = {}, num_nodes = {}, chosen_idx = {} ".format(
        frozen_trial.number,
        frozen_trial.params,
        frozen_trial.user_attrs['r'],
        frozen_trial.user_attrs['num_nodes'],
        frozen_trial.user_attrs['chosen_idx']
        )
    )

study = optuna.create_study(directions=["minimize", "maximize"])  # Create a new study.
study.optimize(objective, n_trials=100, callbacks=[logging_callback])  # Invoke optimization of the objective function.